In [1]:
import os
%run Request_utils.ipynb

In [138]:
MAIN_PAGE_LINK = "https://www.boxofficemojo.com/"
BASIC_PATH = "https://www.boxofficemojo.com"
SEARCH_QUERY = "search/?q="
DIV_SEARCH_CLASS_NAME= "a-fixed-left-grid-col a-col-right"
DIV_DESCRIPTION_CLASS = "a-fixed-left-grid-col a-col-right"
SPAN_DESCRIPTAION_CLASS = "a-size-medium"
DIV_TABLE_DETAILS_CLASS = "a-section a-spacing-none mojo-summary-values mojo-hidden-from-mobile"
DIV_MIDDLE_BAR_TITLE = "mojo-link-bar-internal mojo-flex mojo-flex-h"
DIV_PREFORMANCE_CLASS = "a-section mojo-h-scroll"
CREW_TABLE_ID = "principalCrew"
CAST_TABLE_ID = "principalCast"
DIV_ORIGINAL_RELEASE_DROP_CLASS = "a-section a-spacing-none mojo-dropdown-clear"
TABLE_ROW_TITLE_EARLIESt_RELEASE_DATE = "Earliest Release Date"
TABLE_ROW_TITLE_MPAA = "MPAA"
TABLE_ROW_TITLE_RUNNING_TIME = "Running Time"
TABLE_ROW_TITLE_GENRES = "Genres"
TABLE_ROW_TITLE_BUDGET = "Budget"
MIDDLE_BAR_PERFORMANCE_VALUE = "?ref_=bo_tt_tab"
MIDDLE_BAR_CAST_AND_CREW_VALUE = "credits/?ref_=bo_tt_tab"


In [9]:
def search_movie(name,year):
    url = MAIN_PAGE_LINK + SEARCH_QUERY + name
    webpage = get_url_page(url)
    soup = load_soup_object(webpage)
    search_results__list = soup.find_all("div", {"class": DIV_SEARCH_CLASS_NAME})
    for result in search_results__list:
        span = result.find('span').text
        link = result.find('a')
        if(year in span):
            return (BASIC_PATH + link.get('href'))
    return None
    

In [164]:
def get_movie_description(soup_object):
    div = soup_object.find("div", {"class": DIV_DESCRIPTION_CLASS})
    return div.find('span',{"class":SPAN_DESCRIPTAION_CLASS}).text

def get_relese_date(soup_object):
    temp_date = get_value_from_movie_table(soup_object,TABLE_ROW_TITLE_EARLIESt_RELEASE_DATE)
    return get_date_from_string(temp_date)

def get_mpaa(soup_object):
    return get_value_from_movie_table(soup_object,TABLE_ROW_TITLE_MPAA)

def get_budget(soup_object):
    return get_value_from_movie_table(soup_object,TABLE_ROW_TITLE_BUDGET)

def get_running_time(soup_object):
    temp_time = get_value_from_movie_table(soup_object,TABLE_ROW_TITLE_RUNNING_TIME)
    return convert_string_to_minutes(temp_time)

def get_genres(soup_object):
    ganres = get_value_from_movie_table(soup_object,TABLE_ROW_TITLE_GENRES)
    return ganres.split()

def get_preformance(soup_object):
    link = get_original_release_page_link(soup_object)
    webpage = get_url_page(link)
    new_soup_obj = load_soup_object(webpage)
    div = new_soup.find("div",{"class",DIV_PREFORMANCE_CLASS})
    tr_list = div.find_all("tr")
    data = list()
    for tr in tr_list:
        item = list()
        td_list = tr.find_all("td")
        if(td_list):
            for td in td_list:
                item.append(td.text)
            data.append(item)
    return data

def get_crew_or_cast(soup_object,table_id):
    link = get_middle_bar_links(soup_object,MIDDLE_BAR_CAST_AND_CREW_VALUE)
    webpage = get_url_page(link)
    new_soup_obj = load_soup_object(webpage)
    table = new_soup_obj.find("table",{"id": table_id})
    tr_list = table.find_all("tr")
    crew_or_cast_list = list()
    flag = False
    for tr in tr_list:
        if flag == False:
            flag = True
            continue
        td_list = tr.find_all("td")
        name = remove_empty_lines(td_list[0].find('a').text)
        role = td_list[1].text
        crew_or_cast_list.append([name,role])
    return crew_or_cast_list

def get_value_from_movie_table(soup_object,row_name):
    div = soup_object.find("div",{"class":DIV_TABLE_DETAILS_CLASS})
    table_rows = div.find_all('div')
    for row in table_rows:
        span_list = row.find_all('span')
        if(span_list[0].text == row_name):
            return span_list[1].text
    return None

def get_middle_bar_links(soup_object,match_value):
    div = soup_object.find("div",{"class": DIV_MIDDLE_BAR_TITLE})
    a_list = div.find_all('a')
    for a in a_list:
        if match_value in a.get("href"):
            return BASIC_PATH + a.get("href")
    return None


def get_original_release_page_link(soup_object):
    div = soup_object.find('div',{"class": DIV_ORIGINAL_RELEASE_DROP_CLASS})
    options = div.find_all('option')
    for option in options:
        if option.text == "Original Release":
            return (BASIC_PATH + option.get('value'))   
    return None
    


In [63]:
#utilites
from dateutil import parser
def get_date_from_string(test_str):
    res = parser.parse(test_str, fuzzy=True)
    return res

def convert_string_to_minutes(str_text):
    split_string = str_text.split()
    minutes = (int(split_string[0])*60) + int(split_string[2])
    return minutes

def remove_empty_lines(string):
    lines = string.split("\n")
    non_empty_lines = [line for line in lines if line.strip() != ""]
    return non_empty_lines